In [1]:
# Data manipulation
import numpy as np
import pandas as pd
from math import *
import seaborn as sns
import networkx as nx
import osmnx as ox
import folium

# Visualization.
import matplotlib.pyplot as plt

# Saving models
from datetime import datetime
import joblib

# Display all columns
pd.set_option('display.max_columns', 150,
             'display.max_rows', 150)

# ML
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, RepeatedKFold, train_test_split, cross_validate, cross_val_score 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, median_absolute_error
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn import ensemble

import requests, json
from sqlalchemy import create_engine

In [2]:
full = pd.read_csv("/Users/aurelianosancho/Documents/GitHub/teste_cornershop/full.csv")

In [3]:
full.columns

Index(['Unnamed: 0', 'order_id', 'lat_destination', 'lng_destination',
       'promised_time', 'on_demand', 'shopper_id', 'store_branch_id',
       'total_minutes', 'quantity_UN', 'quantity_KG', 'is_more_UN',
       'UN_plus_KG', 'UN_mult_KG', 'seniority', 'found_rate', 'picking_speed',
       'accepted_rate', 'rating', 'store_id', 'lat_origin', 'lng_origin',
       'Hour', 'Month', 'Day_of_Week', 'Year', 'Date', 'period',
       'distance_havesine', 'distance_car'],
      dtype='object')

In [4]:
var_cat = ['on_demand', 'seniority', 'Hour', 'Day_of_Week', 'period']
var_num = ['quantity_UN', 'quantity_KG', 'UN_plus_KG', 'UN_mult_KG',
           'distance_havesine',
           'found_rate', 'picking_speed','accepted_rate', 'rating',
           'distance_car', 'weight_car', 'duration'
          ]

In [5]:
full = pd.get_dummies(full, columns = var_cat)

In [6]:
cdrop = ['order_id', 'shopper_id', 'store_branch_id', 'store_id',
         'Month', 'Year', 'Date']

In [7]:
full = full.drop(cdrop, axis = 1)

In [8]:
var = ['total_minutes', 'quantity_UN',
       'quantity_KG', 'is_more_UN', 'UN_plus_KG', 'UN_mult_KG', 'found_rate',
       'picking_speed', 'accepted_rate', 'rating',
       'distance_havesine', 
       'on_demand_False', 'on_demand_True',
       'seniority_41dc7c9e385c4d2b6c1f7836973951bf',
       'seniority_50e13ee63f086c2fe84229348bc91b5b',
       'seniority_6c90661e6d2c7579f5ce337c3391dbb9',
       'seniority_bb29b8d0d196b5db5a5350e5e3ae2b1f', 'Hour_0', 'Hour_1',
       'Hour_2', 'Hour_3', 'Hour_11', 'Hour_12', 'Hour_13', 'Hour_14',
       'Hour_15', 'Hour_16', 'Hour_17', 'Hour_18', 'Hour_19', 'Hour_20',
       'Hour_21', 'Hour_22', 'Hour_23', 'Day_of_Week_4', 'Day_of_Week_5',
       'Day_of_Week_6', 'period_afternoon', 'period_dawn', 'period_morning',
       'period_night',
       'distance_car', 'weight_car', 'duration',
       'same_city', 'same_state', 'same_county', 'same_neighbourhood']

full = full[var]

### Out of Sample

In [9]:
ofs = full[full.total_minutes.isnull()]

In [10]:
ofs.head(3)

,total_minutes,quantity_UN,quantity_KG,is_more_UN,UN_plus_KG,UN_mult_KG,found_rate,picking_speed,accepted_rate,rating,distance_havesine,on_demand_False,on_demand_True,seniority_41dc7c9e385c4d2b6c1f7836973951bf,seniority_50e13ee63f086c2fe84229348bc91b5b,seniority_6c90661e6d2c7579f5ce337c3391dbb9,seniority_bb29b8d0d196b5db5a5350e5e3ae2b1f,Hour_0,Hour_1,Hour_2,Hour_3,Hour_11,Hour_12,Hour_13,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,Day_of_Week_4,Day_of_Week_5,Day_of_Week_6,period_afternoon,period_dawn,period_morning,period_night,distance_car
2,NaN,18.0,0.000,1.0,18.000,0.000,0.8313,2.57,0.76,4.92,2.358128,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,2930.5
5,NaN,15.0,0.000,1.0,15.000,0.000,0.8946,1.82,1.00,4.84,4.190793,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,5786.8
9,NaN,94.0,2.852,1.0,96.852,268.088,0.9363,1.67,1.00,4.84,2.512359,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,3141.8


In [11]:
ofs.isnull().sum()

total_minutes                                 1995
quantity_UN                                      0
quantity_KG                                      0
is_more_UN                                       0
UN_plus_KG                                       0
UN_mult_KG                                       0
found_rate                                       0
picking_speed                                    0
accepted_rate                                    0
rating                                           0
distance_havesine                                0
on_demand_False                                  0
on_demand_True                                   0
seniority_41dc7c9e385c4d2b6c1f7836973951bf       0
seniority_50e13ee63f086c2fe84229348bc91b5b       0
seniority_6c90661e6d2c7579f5ce337c3391dbb9       0
seniority_bb29b8d0d196b5db5a5350e5e3ae2b1f       0
Hour_0                                           0
Hour_1                                           0
Hour_2                         

### Modeling data

In [12]:
modeling = full[~full.total_minutes.isnull()]

In [13]:
modeling.head(3)

,total_minutes,quantity_UN,quantity_KG,is_more_UN,UN_plus_KG,UN_mult_KG,found_rate,picking_speed,accepted_rate,rating,distance_havesine,on_demand_False,on_demand_True,seniority_41dc7c9e385c4d2b6c1f7836973951bf,seniority_50e13ee63f086c2fe84229348bc91b5b,seniority_6c90661e6d2c7579f5ce337c3391dbb9,seniority_bb29b8d0d196b5db5a5350e5e3ae2b1f,Hour_0,Hour_1,Hour_2,Hour_3,Hour_11,Hour_12,Hour_13,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,Day_of_Week_4,Day_of_Week_5,Day_of_Week_6,period_afternoon,period_dawn,period_morning,period_night,distance_car
0,67.684264,16.0,2.756,1.0,18.756,44.096,0.9024,1.30,0.92,4.76,1.823597,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,3367.1
1,57.060632,11.0,0.000,1.0,11.000,0.000,0.7610,2.54,0.92,4.96,1.935026,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,2373.9
3,52.067742,1.0,0.000,1.0,1.000,0.000,0.8776,2.80,0.96,4.76,3.820244,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,5632.1


In [14]:
modeling.isnull().sum()

total_minutes                                 0
quantity_UN                                   0
quantity_KG                                   0
is_more_UN                                    0
UN_plus_KG                                    0
UN_mult_KG                                    0
found_rate                                    0
picking_speed                                 0
accepted_rate                                 0
rating                                        0
distance_havesine                             0
on_demand_False                               0
on_demand_True                                0
seniority_41dc7c9e385c4d2b6c1f7836973951bf    0
seniority_50e13ee63f086c2fe84229348bc91b5b    0
seniority_6c90661e6d2c7579f5ce337c3391dbb9    0
seniority_bb29b8d0d196b5db5a5350e5e3ae2b1f    0
Hour_0                                        0
Hour_1                                        0
Hour_2                                        0
Hour_3                                  

In [15]:
# Getting the data:
X = modeling.drop(['total_minutes'], axis = 1)

y = modeling['total_minutes']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 451, test_size= .25)

In [17]:
StdSca = StandardScaler() ## or standerscaler 
X_train[var_num] = pd.DataFrame(StdSca.fit_transform(X_train[var_num]), columns = var_num, index = X_train.index)
X_test[var_num] = pd.DataFrame(StdSca.transform(X_test[var_num]), columns = var_num, index = X_test.index)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [18]:
ofs[var_num] = StdSca.transform(ofs[var_num])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [23]:
X_train.columns

Index(['quantity_UN', 'quantity_KG', 'is_more_UN', 'UN_plus_KG', 'UN_mult_KG',
       'found_rate', 'picking_speed', 'accepted_rate', 'rating',
       'distance_havesine', 'on_demand_False', 'on_demand_True',
       'seniority_41dc7c9e385c4d2b6c1f7836973951bf',
       'seniority_50e13ee63f086c2fe84229348bc91b5b',
       'seniority_6c90661e6d2c7579f5ce337c3391dbb9',
       'seniority_bb29b8d0d196b5db5a5350e5e3ae2b1f', 'Hour_0', 'Hour_1',
       'Hour_2', 'Hour_3', 'Hour_11', 'Hour_12', 'Hour_13', 'Hour_14',
       'Hour_15', 'Hour_16', 'Hour_17', 'Hour_18', 'Hour_19', 'Hour_20',
       'Hour_21', 'Hour_22', 'Hour_23', 'Day_of_Week_4', 'Day_of_Week_5',
       'Day_of_Week_6', 'period_afternoon', 'period_dawn', 'period_morning',
       'period_night', 'distance_car'],
      dtype='object')

## Model

In [19]:
CV = cross_validate(estimator = LinearRegression(), 
                    cv = 5, 
                    #scoring = "r2", 
                    scoring = ["r2", "neg_median_absolute_error"], 
                    X = X_train,
                    y = y_train)

cv_df = pd.DataFrame(CV)
cv_df

,fit_time,score_time,test_r2,test_neg_median_absolute_error
0,0.551753,0.003559,0.409297,-16.743869
1,0.011834,0.004728,0.349139,-15.877344
2,0.012669,0.005967,0.382416,-16.411225
3,0.015127,0.005468,0.434951,-16.406138
4,0.014683,0.006510,0.479484,-16.121321


In [20]:
CV = cross_validate(estimator = ElasticNet(alpha=0.025, l1_ratio=0.5), 
                    cv = 5, 
                    #scoring = "r2", 
                    scoring = ["r2", "neg_median_absolute_error"], 
                    X = X_train,
                    y = y_train)

cv_df = pd.DataFrame(CV)
cv_df

,fit_time,score_time,test_r2,test_neg_median_absolute_error
0,0.081175,0.005212,0.406735,-16.584178
1,0.069476,0.004162,0.347305,-15.861442
2,0.065701,0.004048,0.380647,-16.236324
3,0.065833,0.004606,0.432348,-16.537199
4,0.067589,0.004627,0.478334,-16.414390


In [21]:
model = ElasticNet(alpha=1.0, l1_ratio=0.5).fit(X = X_train, y = y_train)

In [22]:
ofs = ofs.drop(['total_minutes'], axis = 1)
predict = model.predict(ofs)